In [73]:
import pandas as pd

def import_data():
    gtfs_dir = '/Users/denisvoroncov/Desktop/Project/Vienna Linien — Timetable data GTFS Vienna/data.gv.at/'

    # Load data
    agency_df = pd.read_csv(gtfs_dir + 'GTFS Agency-6.plain')
    stops_df = pd.read_csv(gtfs_dir + 'GTFS Stops-9.plain')
    routes_df = pd.read_csv(gtfs_dir + 'GTFS Routes-5.plain')
    trips_df = pd.read_csv(gtfs_dir + 'GTFS Trips-7.plain')
    stop_times_df = pd.read_csv(gtfs_dir + 'GTFS Stop Times-8.plain')
    calendar_df = pd.read_csv(gtfs_dir + 'GTFS Calendar-2.plain')
    calendar_dates_df = pd.read_csv(gtfs_dir + 'GTFS Calendar Dates-3.plain')
    shapes_df = pd.read_csv(gtfs_dir + 'GTFS Shapes-1.plain')
    
    # Return all DataFrames
    return agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df, calendar_dates_df, shapes_df

# Import and display data
agency, stops, routes, trips, stop_times, calendar, calendar_dates, shapes = import_data()

In [74]:
import scipy.stats as stats


# Transfer reliability 

def calculate_transfer_probability(scheduled_arrival, next_bus_departure, shape=2, scale=3):
    
    transfer_window = next_bus_departure - scheduled_arrival
    if transfer_window < 0:
        return 0.0  # No chance of a successful transfer if arrival is after departure
    return stats.gamma.cdf(transfer_window, a=shape, scale=scale)

In [149]:
import sys
import heapq
from collections import defaultdict
from datetime import datetime, timedelta
from scipy.stats import norm

# Hilfsfunktion: Zeit in Minuten umwandeln
def time_to_minutes(time_str):
    hours, minutes, seconds = map(int, time_str.split(":"))
    return hours * 60 + minutes + seconds / 60

def minutes_to_time(minutes):
    hours = int(minutes // 60)
    minutes = int(minutes % 60)
    return f"{hours:02d}:{minutes:02d}"

def get_weekday(date):
    weekdays = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    return weekdays[date.weekday()]

from datetime import datetime


#FFFFFFFIIIIIIIIXXXXXXEEEEEEDDDDDD

def is_service_available(service_id, date, calendar, calendar_dates):

    # Convert date to string in YYYYMMDD format
    date_str = date.strftime("%Y%m%d")
    weekday = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"][date.weekday()]

    # Step 1: Check for exceptions in calendar_dates
    if service_id in calendar_dates["service_id"].values:
        exceptions = calendar_dates[calendar_dates["service_id"] == service_id]
        for _, exception in exceptions.iterrows():
            if exception["date"] == int(date_str):
                if exception["exception_type"] == 2:  # Service is added as an exception
                    return True
                elif exception["exception_type"] == 1:  # Service is removed as an exception
                    return False

    # Step 2: Check for regular service in calendar
    if service_id in calendar["service_id"].values:
        service = calendar[calendar["service_id"] == service_id]
        # Check if date is within start_date and end_date
        if int(service["start_date"].iloc[0]) <= int(date_str) <= int(service["end_date"].iloc[0]):
            # Check if the service operates on this weekday
            is_available = (service[weekday].iloc[0] == 1)
            return is_available

def identify_transfer_stops(stop_times, trips, date, calendar, calendar_dates): 
    """
    Identify transfer stops based on unique routes operating at a stop.
    Assumes no NaN values in the dataset.
    """

    # Map trip_id to route_id and service_id
    trip_to_route = trips.set_index("trip_id")["route_id"].to_dict()
    trip_to_service = trips.set_index("trip_id")["service_id"].to_dict()

    # Add route_id and service_id to stop_times
    stop_times = stop_times.copy()
    stop_times["route_id"] = stop_times["trip_id"].map(trip_to_route)
    stop_times["service_id"] = stop_times["trip_id"].map(trip_to_service)

    # Precompute service availability
    unique_service_ids = stop_times["service_id"].unique()
    service_availability = {
        service_id: is_service_available(service_id, date, calendar, calendar_dates)
        for service_id in unique_service_ids
    }

    # Filter stop_times for active services
    stop_times = stop_times[stop_times["service_id"].map(service_availability)]

    # Group by stop_id and count unique route_ids
    stop_route_counts = stop_times.groupby("stop_id")["route_id"].nunique()

    # Identify stops with more than one route
    transfer_stops = set(stop_route_counts[stop_route_counts > 1].index)

    return transfer_stops


# Simplify stop_times by collapsing non-transfer stops
def simplify_network_with_no_transfer_stops(start_stop_name, end_stop_name, stop_times, trips, transfer_stops, date, calendar, calendar_dates, stops): 
    
    
    # RUNTIME TO BE ASSESSED
    # Robust mapping from stop name to stop ID
    if start_stop_name not in stops["stop_name"].values:
        raise ValueError(f"Start stop name '{start_stop_name}' not found in stops DataFrame.")
    if end_stop_name not in stops["stop_name"].values:
        raise ValueError(f"End stop name '{end_stop_name}' not found in stops DataFrame.")

    stop_name_to_stop_id = stops.set_index("stop_name")["stop_id"].to_dict()
    start_stop_id = stop_name_to_stop_id[start_stop_name]
    end_stop_id = stop_name_to_stop_id[end_stop_name]


    trip_id_to_service = trips.set_index("trip_id")["service_id"].to_dict()

    simplified_rows = []
    grouped = stop_times.groupby("trip_id")
    
    for trip_id, group in grouped: # group of stop in a particular trip
        service_id = trip_id_to_service[trip_id]
        if not is_service_available(service_id, date, calendar, calendar_dates):
            continue
        

        group = group.sort_values("stop_sequence")

        for _, row in group.iterrows(): # iterate through each row of sorted stops in a trip
            stop_id = row["stop_id"]

            if stop_id in transfer_stops or stop_id == start_stop_id or stop_id == end_stop_id:  # Handle transfer stops + start point + destination point
                # Add the current collapsed segment to the result
                simplified_rows.append(row)

    # Convert the simplified rows into a DataFrame
    simplified_stop_times = pd.DataFrame(simplified_rows)

    # Reset stop_sequence values to be consecutive for each trip_id
    simplified_stop_times["stop_sequence"] = simplified_stop_times.groupby("trip_id").cumcount() + 1

    return simplified_stop_times


In [155]:
def prepare_network(
    start_stop_name, 
    end_stop_name, 
    start_datetime, 
    time_budget_minutes, 
    stop_times, 
    trips, 
    calendar, 
    calendar_dates, 
    stops
):
    """
    Prepare the network by filtering stop_times based on the start time, time budget,
    and active services on the given date.
    """
    print(f"Rows before time window filter: {len(stop_times)}")

    # Parse the start datetime and calculate time window
    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    end_time_obj = start_time_obj + timedelta(minutes=time_budget_minutes)
    date = start_time_obj.date()
    stop_times_copy = stop_times.copy()
    # Filter out stops outside the time window
    stop_times_copy["arrival_minutes"] = stop_times_copy["arrival_time"].apply(time_to_minutes)
    stop_times_copy["departure_minutes"] = stop_times_copy["departure_time"].apply(time_to_minutes)

    start_minutes = time_to_minutes(start_time_obj.strftime("%H:%M:%S"))
    end_minutes = time_to_minutes(end_time_obj.strftime("%H:%M:%S"))

    stop_times_copy = stop_times_copy[
        (stop_times_copy["arrival_minutes"] >= start_minutes) &
        (stop_times_copy["departure_minutes"] <= end_minutes)
    ]

    print(f"Rows after time window filter: {len(stop_times)}")

    # Precompute service availability
    trip_id_to_service = trips.set_index("trip_id")["service_id"].to_dict()
    # Ensure it's a new DataFrame, not a slice
    stop_times_copy["service_id"] = stop_times_copy["trip_id"].map(trip_id_to_service)

    unique_service_ids = stop_times_copy["service_id"].dropna().unique()
    service_availability = {
        service_id: is_service_available(service_id, date, calendar, calendar_dates)
        for service_id in unique_service_ids
    }

    # Count rows before availability filter
    before_filter_count = len(stop_times)

    # Filter using precomputed availability
    # Use .map() with a default value directly to avoid NaN
    # Filter using precomputed availability
    stop_times_copy = stop_times_copy[
    stop_times_copy["service_id"].map(lambda x: service_availability.get(x, False)).astype(bool)
    ]

    # Count rows after availability filter
    after_filter_count = len(stop_times_copy)

    print(f"Rows before availability filter: {before_filter_count}")
    print(f"Rows after availability filter: {after_filter_count}")

    # Identify transfer stops
    transfer_stops = identify_transfer_stops(stop_times_copy, trips, date, calendar, calendar_dates)

    # Simplify the network (use your existing logic here)
    # Example:
    simplified_stop_times = simplify_network_with_no_transfer_stops(
        start_stop_name, 
        end_stop_name, 
        stop_times_copy, 
        trips, 
        transfer_stops, 
        date, 
        calendar, 
        calendar_dates,
        stops
    )
    print(f"Simplified Stop Times: {len(simplified_stop_times)} rows remaining.")
    
    return simplified_stop_times

In [142]:
def create_graph_with_schedule(stop_times, stops, trips):
    

    
    graph = defaultdict(list)
    
    stop_id_to_name = stops.set_index("stop_id")["stop_name"].to_dict()
    trip_id_to_route = trips.set_index("trip_id")["route_id"].to_dict()

    # Sortiere stop_times nach Trip und Stop-Sequence
    stop_times = stop_times.sort_values(by=["trip_id", "stop_sequence"])
    grouped = stop_times.groupby("trip_id")

    for trip_id, group in grouped:

        stops_in_trip = group["stop_id"].tolist()
        arrival_times = group["arrival_time"].tolist()
        departure_times = group["departure_time"].tolist()

        # Füge Verbindungen zwischen aufeinanderfolgenden Haltestellen hinzu
        for i in range(len(stops_in_trip) - 1):
            start_stop_id = stops_in_trip[i]
            end_stop_id = stops_in_trip[i + 1]

            start_departure = time_to_minutes(departure_times[i])
            end_arrival = time_to_minutes(arrival_times[i + 1])

            travel_time = end_arrival - start_departure  # Dauer in Minuten

            if travel_time > 0:  # Vermeide ungültige Zeiten
                start_stop_name = stop_id_to_name[start_stop_id]
                end_stop_name = stop_id_to_name[end_stop_id]
                route_id = trip_id_to_route[trip_id]  # Hole die Route/Linie

                graph[start_stop_name].append((end_stop_name, start_departure, end_arrival, route_id))

    return graph

In [156]:
# Define input parameters
start_stop_name = "Pötzleinsdorf"
end_stop_name = "Konstanziagasse"
start_datetime = "2025-10-16 14:30:00"
time_budget_minutes = 120  # 2 hours

# Run the `prepare_network` function
simplified_stop_times = prepare_network(
    start_stop_name,
    end_stop_name,
    start_datetime,
    time_budget_minutes,
    stop_times,
    trips,
    calendar,
    calendar_dates,
    stops
)

# Output the simplified stop_times DataFrame
print(simplified_stop_times)

Rows before time window filter: 3159655
Rows after time window filter: 3159655
Rows before availability filter: 3159655
Rows after availability filter: 129589
Simplified Stop Times: 47104 rows remaining.
                        trip_id arrival_time departure_time         stop_id  \
1499452   1.T9.23-50A-j25-1.1.H     15:58:00       15:58:00   at:49:123:0:3   
1499453   1.T9.23-50A-j25-1.1.H     15:59:00       15:59:00   at:49:184:0:2   
1499454   1.T9.23-50A-j25-1.1.H     16:00:00       16:00:00  at:49:1501:0:1   
1499463   1.T9.23-50A-j25-1.1.H     16:15:00       16:15:00   at:49:323:0:2   
3088361  10.T0.11-WLB-j25-2.4.R     14:39:00       14:39:00  at:49:1015:0:5   
...                         ...          ...            ...             ...   
3069034  99.TC.23-35A-j25-1.3.H     15:57:00       15:57:00  at:49:1462:0:1   
3069035  99.TC.23-35A-j25-1.3.H     15:59:00       15:59:00  at:49:1509:0:2   
3069037  99.TC.23-35A-j25-1.3.H     16:02:00       16:02:00   at:49:376:0:4   
306904

In [157]:
# Fix für Dijkstra-Algorithmus mit Umstiegsoptimierung
def dijkstra_with_reliability_fixed(graph, start_name, end_name, start_time_minutes):
    pq = [(start_time_minutes, start_name, [], 1.0, None)]
    visited = set()

    while pq:
        current_time, current_stop, path, reliability, last_route = heapq.heappop(pq)

        if (current_stop, current_time) in visited:
            continue
        visited.add((current_stop, current_time))

        path = path + [(current_stop, current_time)]
        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time:
                transfer_reliability = 1.0 if last_route == route_id else calculate_transfer_probability(arrival_time, departure_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability
                heapq.heappush(pq, (
                    new_current_time, neighbor, path + [(route_id, departure_time, arrival_time)], new_reliability, route_id))

        if current_stop == end_name:
            return current_time, path, reliability

    return float("inf"), [], 0.0

The transfer probability is calculated using cdf, but we say there are not depature delays. If there are, reliability only goes higher, for all itineraries, we want the most extreme case however. Furthermore, it is simply easier to calculate. 

In [158]:
# Hauptprogramm
if __name__ == "__main__":
    start_stop_name = "Pötzleinsdorf"
    end_stop_name = "Konstanziagasse"
    start_datetime = "2025-10-16 14:30:00"
    time_budget_minutes = 120

    #agency, stops, routes, trips, stop_times, calendar, calendar_dates = import_data()

    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    date = start_time_obj.date()
    start_time_minutes = start_time_obj.hour * 60 + start_time_obj.minute

    simplified_stop_times = prepare_network(
        start_stop_name, 
        end_stop_name, 
        start_datetime, 
        time_budget_minutes, 
        stop_times, 
        trips, 
        calendar, 
        calendar_dates, 
        stops
    )

    graph = create_graph_with_schedule(simplified_stop_times, stops, trips)

    if start_stop_name not in graph or end_stop_name not in graph:
        print("🚨 Ungültige Start- oder Zielhaltestelle!")
        sys.exit()

    # Finde optimierte Route
    arrival_time_minutes_fixed, path_fixed, reliability_fixed = dijkstra_with_reliability_fixed(
        graph, start_stop_name, end_stop_name, start_time_minutes
    )

    # Ergebnis anzeigen
    if arrival_time_minutes_fixed < float("inf"):
        arrival_time_fixed = minutes_to_time(arrival_time_minutes_fixed)
        print(f"\n📍 Optimierte zuverlässigste Route von {start_stop_name} nach {end_stop_name}:")

        last_route = None
        grouped_routes = []

        for i in range(1, len(path_fixed) - 1, 2):
            current_stop, current_time = path_fixed[i-1]
            route_id, departure_time, arrival_time = path_fixed[i]
            next_stop, _ = path_fixed[i + 1]

            if route_id == last_route:
                grouped_routes[-1]["stops"].append((next_stop, arrival_time))
            else:
                grouped_routes.append({
                    "route_id": route_id,
                    "start_stop": current_stop,
                    "departure_time": departure_time,
                    "stops": [(next_stop, arrival_time)]
                })
            last_route = route_id

        for segment in grouped_routes:
            start = segment["start_stop"]
            dep_time = minutes_to_time(segment["departure_time"])
            route = segment["route_id"]
            stops = " → ".join([f"{stop} (Ankunft: {minutes_to_time(arr)})" for stop, arr in segment["stops"]])
            print(f"  🚆 {start} (Abfahrt: {dep_time}) → {stops} mit Linie {route}")

        print(f"\n🎯 Endstation: {end_stop_name} (Ankunft: {arrival_time_fixed})")
        print(f"🔹 Gesamt-Zuverlässigkeit der Route: {reliability_fixed:.2f}\n")
    else:
        print(f"\n⚠️ Keine zuverlässige Route von {start_stop_name} nach {end_stop_name} gefunden.\n")

Rows before time window filter: 3159655
Rows after time window filter: 3159655
Rows before availability filter: 3159655
Rows after availability filter: 129589
Simplified Stop Times: 47104 rows remaining.

📍 Optimierte zuverlässigste Route von Pötzleinsdorf nach Konstanziagasse:
  🚆 Pötzleinsdorf (Abfahrt: 14:36) → Gersthof S (Ankunft: 14:43) mit Linie 22-41-j25-1
  🚆 Gersthof S (Abfahrt: 14:48) → Weinhauser Gasse (Ankunft: 14:49) → Aumannplatz (Ankunft: 14:50) → Martinstraße (Ankunft: 14:52) → Kutschkergasse (Ankunft: 14:53) → Währinger Str.-Volksoper U (Ankunft: 14:55) → Spitalgasse (Ankunft: 14:57) mit Linie 22-40-j25-1
  🚆 Spitalgasse (Abfahrt: 14:58) → Nußdorfer Straße/Alserbachstraße (Ankunft: 15:00) → Franz-Josefs-Bahnhof S (Ankunft: 15:02) → Friedensbrücke U (Ankunft: 15:03) → Klosterneuburger Straße/Wallensteinstraße (Ankunft: 15:04) → Wallensteinplatz (Ankunft: 15:05) → Höchstädtplatz (Ankunft: 15:10) mit Linie 22-33-j25-1
  🚆 Höchstädtplatz (Abfahrt: 15:11) → Floridsdorf S+U 